# Finding Causality in Big Data

## Import required libraries and data

In [3]:
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import binom, hypergeom, gaussian_kde
from scipy.stats import ttest_ind
import math
from numpy import linalg
import scipy.spatial.distance as ssdist
import statsmodels.genmod.generalized_linear_model as sm

# Attributes:

* school - student's school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)
* sex - student's sex (binary: "F" - female or "M" - male)
* age - student's age (numeric: from 15 to 22)
* address - student's home address type (binary: "U" - urban or "R" - rural)
* famsize - family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)
* Pstatus - parent's cohabitation status (binary: "T" - living together or "A" - apart)
* Medu - mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
* Fedu - father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
* Mjob - mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
* Fjob - father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
* reason - reason to choose this school (nominal: close to "home", school "reputation", "course" preference or "other")
* guardian - student's guardian (nominal: "mother", "father" or "other")
* traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
* studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
* failures - number of past class failures (numeric: n if 1<=n<3, else 4)
* schoolsup - extra educational support (binary: yes or no)
* famsup - family educational support (binary: yes or no)
* paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
* activities - extra-curricular activities (binary: yes or no)
* nursery - attended nursery school (binary: yes or no)
* higher - wants to take higher education (binary: yes or no)
* internet - Internet access at home (binary: yes or no)
* romantic - with a romantic relationship (binary: yes or no)
* famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
* freetime - free time after school (numeric: from 1 - very low to 5 - very high)
* goout - going out with friends (numeric: from 1 - very low to 5 - very high)
* Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
* Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
* health - current health status (numeric: from 1 - very bad to 5 - very good)
* absences - number of school absences (numeric: from 0 to 93)

### These grades are related with the course subject, Math or Portuguese:
* G1 - first period grade (numeric: from 0 to 20)
* G2 - second period grade (numeric: from 0 to 20)
* G3 - final grade (numeric: from 0 to 20, output target)

In [5]:
# 1. read data
df = sqlContext.sql("SELECT * FROM student_mat").toPandas()
df = df.drop(['G1','G2'],1)
print(df.columns)
print(df.shape) # 382 students
df.head()

In [6]:
# 2. convert all attributes into integer
# to do: store mappings for each attribute in a dictionary
for col in df:
    if (df[col].dtypes == object) == True:
        new_map = dict(zip(df[col].unique(),range(len(df[col].unique()))))
        df[col] = df[col].map(new_map)
df.head()

* Outcome: Final grade
* Treatment: paid - extra paid classes within the course subject (binary: yes or no)
* Covariates: all the other attributes

## Define treatment, outcome and covariates

In [9]:
df.head()

In [10]:
# treatment
TREAT = 'paid'
Tr = df[TREAT]

# outcome
OUT = 'G3'

# 1 Identify covariates from observational data
## 1.1 Difference-in-means: outcome variable

* Effectiveness of treatment (effect on RE78 expression)
* Question: does treatment affect RE78 expression (outcome)?

In [12]:
df.groupby(TREAT).size()

In [13]:
df.groupby(TREAT)[OUT].mean().reset_index()

In [14]:
# t-statistic & p-value for difference in outcome of two groups
def hypothesisTest(data, attribute, group):
    x = data[attribute][df[group]== 1]
    y = data[attribute][df[group]== 0]
    # t-statistic & p-value for difference in outcome of two groups
    t = ttest_ind(x, y)[0]
    p = ttest_ind(x, y)[1]
    # Confidence intervals
    cm = sms.CompareMeans(sms.DescrStatsW(x), sms.DescrStatsW(y))
    CI = str(round(cm.tconfint_diff(usevar='unequal')[0]))+' - '+str(round(cm.tconfint_diff(usevar='unequal')[1]))
    # Cohen's d
    pooledvar = math.sqrt((pow(x.std(),2) + (pow(y.std(),2)))/2)
    d = (x.mean()-y.mean()) / pooledvar
    # create dataframe
    tablelist = []
    tablerow = [attribute,x.mean()-y.mean(),t,p,CI,d]
    tablelist.append(tablerow)
    out = pd.DataFrame(tablelist)
    out.columns = ['Attribute','Mean Difference','t-value','p-value','95% Confidence Intervals',"Cohen's d"]

    return out

In [15]:
# covariates
b = pd.DataFrame()
for att in df.drop([TREAT,OUT],1):
    a = []
    a = Test_Class.hypothesisTest(df, att, TREAT)
    b = pd.concat([b,a],0)
b.sort('p-value')

In [16]:
# covariates
COV = np.array(b.Attribute[b['p-value'] <= 0.05])
X = df[COV]
X.head()

## 2.1 Propensity score matching

In [18]:
####### Using GLM
glm_binom = sm.GLM(Tr, X, family=sm.families.Binomial())
res = glm_binom.fit()
print res.summary()
propensityScoreGLM = res.fittedvalues
df_new = pd.concat([df[COV],df[OUT],propensityScoreGLM],1)
df_new.columns = np.append(COV, [OUT,'Propensity Score'], axis=None )
print '\n'+ 'Propensity Scores added: '
df_new.head()

## Matching 

There are several variants of matching: 
* one-to-one matching
* one-to-many matching 
* with or without a caliper, and with or without replacement

In [20]:
def Match(groups, propensity, caliper = 0.05, caliper_method = "propensity", replace = False):
        
    # Code groups as 0 and 1
    groups = groups == groups.unique()[0]
    N = len(groups)
    N1 = groups[groups == 1].index; N2 = groups[groups == 0].index
    g1, g2 = propensity[groups == 1], propensity[groups == 0]
    # Check if treatment groups got flipped - the smaller should correspond to N1/g1
    if len(N1) > len(N2):
       N1, N2, g1, g2 = N2, N1, g2, g1
        
    # Randomly permute the smaller group to get order for matching
    morder = np.random.permutation(N1)
    matches = {}
    
    for m in morder:
        dist = abs(g1[m] - g2)
        if (dist.min() <= caliper) or not caliper:
            matches[m] = dist.argmin()    # Potential problem: check for ties
            if not replace:
                g2 = g2.drop(matches[m])
    return (matches)

def MatchMany(groups, propensity, method = "caliper", k = 1, caliper = 0.05, caliper_method = "propensity", replace = True):
    
    # Transform the propensity scores and caliper when caliper_method is "logit" or "none"
    if method == "caliper":
        if caliper_method == "logit":
            propensity = log(propensity/(1-propensity))
            caliper = caliper*np.std(propensity)
        elif caliper_method == "none":
            caliper = 0
    
    # Code groups as 0 and 1
    groups = groups == groups.unique()[0]
    N = len(groups)
    N1 = groups[groups == 1].index; N2 = groups[groups == 0].index
    g1, g2 = propensity[groups == 1], propensity[groups == 0]
    # Check if treatment groups got flipped - the smaller should correspond to N1/g1
    if len(N1) > len(N2):
       N1, N2, g1, g2 = N2, N1, g2, g1
        
        
    # Randomly permute the smaller group to get order for matching
    morder = np.random.permutation(N1)
    matches = {}
    
    for m in morder:
        dist = abs(g1[m] - g2)
        dist.sort()
        if method == "knn":
            caliper = dist.iloc[k-1]
        # PROBLEM: when there are ties in the knn. 
        # Need to randomly select among the observations tied for the farthest eacceptable distance
        keep = np.array(dist[dist<=caliper].index)
        if len(keep):
            matches[m] = keep
        else:
            matches[m] = [dist.argmin()]
        if not replace:
            g2 = g2.drop(matches[m])
    return (matches)  

def whichMatched(matches, data, many = False):
  
    tr = matches.keys()
    if many:
        ctrl = [m for matchset in matches.values() for m in matchset]
    else:
        ctrl = matches.values()
    # need to remove duplicate rows, which may occur in matching with replacement
    temp = pd.concat([data.ix[tr], data.ix[ctrl]])

    return temp

In [21]:
# one-to-one without caliper, without replacement
matches = Match(Tr, propensityScoreGLM, caliper = 0, caliper_method = "propensity", replace = False)
temp = whichMatched(matches, df, many = False)

PS_matched = temp.groupby(temp.index).first()
PS_matched.groupby(TREAT).size().reset_index()

In [22]:
# Exercise: Run one-to-one matching with fixed 0.05 caliper, without replacement
matches = 'one-to-one matching with caliper, with replacement'
temp = 'matched data'

PS_matched1 = temp.groupby(temp.index).first()
PS_matched1.groupby(TREAT).size().reset_index()

In [23]:
# Exercise: Run one-to-one matching with logit caliper, without replacement
matches = 'one-to-one matching with caliper, with replacement'
temp = 'matched data'

PS_matched2 = temp.groupby(temp.index).first()
PS_matched2.groupby(TREAT).size().reset_index()

In [24]:
# Exercise: Run one-to-one matching with logit caliper, with replacement
matches = 'one-to-one matching with caliper, with replacement'
temp = 'matched data'

PS_matched3 = temp.groupby(temp.index).first()
PS_matched3.groupby(TREAT).size().reset_index()

In [25]:
# Exercise: Run one-to-many matching with caliper, with replacement
matches = 'one-to-many matching with logit caliper, with replacement'
temp = 'matched data'

PS_matched4 = temp.groupby(temp.index).first()
PS_matched4.groupby(TREAT).size().reset_index()

In [26]:
# Exercise: Run one-to-many matching with fixed 0.05 caliper, with replacement
matches = 'one-to-many matching with fixed 0.05 caliper, with replacement'
temp = 'matched data'

PS_matched5 = temp.groupby(temp.index).first()
PS_matched5.groupby(TREAT).size().reset_index()

In [27]:
# Exercise: Run one-to-many matching with fixed 0.05 caliper, without replacement
matches = 'one-to-many matching with fixed 0.05 caliper, without replacement'
temp = 'matched data'

PS_matched6 = temp.groupby(temp.index).first()
PS_matched6.groupby(TREAT).size().reset_index()

# 5 Evaluating matching methods

* Normalize each matched attribute to 0-1
* Find the difference of means between treatment and control groups

In [29]:
f = plt.figure(figsize=(15,5))

PS_matched_norm = (PS_matched[COV] - PS_matched[COV].mean()) / (PS_matched[COV].max() - PS_matched[COV].min())
PS_matched_mean = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched[TREAT]==1].mean() - PS_matched_norm[PS_matched[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched1[COV] - PS_matched1[COV].mean()) / (PS_matched1[COV].max() - PS_matched1[COV].min())
PS_matched_mean1 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched1[TREAT]==1].mean() - PS_matched_norm[PS_matched1[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched2[COV] - PS_matched2[COV].mean()) / (PS_matched2[COV].max() - PS_matched2[COV].min())
PS_matched_mean2 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched2[TREAT]==1].mean() - PS_matched_norm[PS_matched2[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched3[COV] - PS_matched3[COV].mean()) / (PS_matched3[COV].max() - PS_matched3[COV].min())
PS_matched_mean3 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched3[TREAT]==1].mean() - PS_matched_norm[PS_matched3[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched4[COV] - PS_matched4[COV].mean()) / (PS_matched4[COV].max() - PS_matched4[COV].min())
PS_matched_mean4 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched4[TREAT]==1].mean() - PS_matched_norm[PS_matched4[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched5[COV] - PS_matched5[COV].mean()) / (PS_matched5[COV].max() - PS_matched5[COV].min())
PS_matched_mean5 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched4[TREAT]==1].mean() - PS_matched_norm[PS_matched4[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched6[COV] - PS_matched6[COV].mean()) / (PS_matched6[COV].max() - PS_matched6[COV].min())
PS_matched_mean6 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched4[TREAT]==1].mean() - PS_matched_norm[PS_matched4[TREAT]==0].mean())}).mean()[0]


x = [PS_matched.shape[0],PS_matched1.shape[0],PS_matched2.shape[0],PS_matched3.shape[0],PS_matched4.shape[0],,PS_matched5.shape[0],,PS_matched6.shape[0]]
y = [PS_matched_mean ,PS_matched_mean1,PS_matched_mean2,PS_matched_mean3,PS_matched_mean4,PS_matched_mean5,PS_matched_mean6]
colors = ['r','b','g','c','k','m','y']
MatchingMethod = ['1-1 wC woutR','1-1 wC woutR','1-1 wC woutR','1-1 wC wR','1-m wC wR','1-m wC wR','1-m wC woutR']

j = 0
for i in x:
    plt.scatter(i, y[j], s=120, c=colors[j], label=MatchingMethod[j])
    j += 1
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),ncol=1, fancybox=True, fontsize=8)
    
plt.xlabel('Sample Size')
plt.ylabel('Balance Metric')

display(f)

## 5 Estimate treatment effects

Estimate average treatment effect  on the treated

### 5.1 Computes ATT using difference in means

In [31]:
# t-statistic & p-value for difference in outcome of two groups
def hypothesisTestOutcome(data, attribute, group):
    x = data[attribute][df[group]== 1]
    y = data[attribute][df[group]== 0]
    # t-statistic & p-value for difference in outcome of two groups
    t = ttest_ind(x, y)[0]
    p = ttest_ind(x, y)[1]
    # Confidence intervals
    cm = sms.CompareMeans(sms.DescrStatsW(x), sms.DescrStatsW(y))
    CI = str(round(cm.tconfint_diff(usevar='unequal')[0]))+' - '+str(round(cm.tconfint_diff(usevar='unequal')[1]))
    # Cohen's d
    pooledvar = math.sqrt((pow(x.std(),2) + (pow(y.std(),2)))/2)
    d = (x.mean()-y.mean()) / pooledvar
    # create dataframe
    tablelist = []
    tablerow = [attribute,x.mean()-y.mean(),((x.mean()-y.mean())/df[attribute].max()),t,p,d]
    tablelist.append(tablerow)
    out = pd.DataFrame(tablelist)
    out.columns = ['Attribute','Outcome Mean Difference','Outcome Mean Difference %','t-value','p-value',"Cohen's d"]

    return out

In [32]:
# Matched outcome
Y0_PS_matched = PS_matched.G3[df.paid == 0]
Y1_PS_matched = PS_matched.G3[df.paid == 1]

# Exercise: compute the difference in means for matched groups
print 'Differene in means: '+ str('compute the difference in means for matched groups')